!pip install opencv-contrib-python


In [39]:
import cv2
import numpy as np



In [40]:
path_input = 'Images/Input/'
path_output = 'Images/Output/'

In [41]:
# Détection de personnes

In [42]:
# Load Yolo
net = cv2.dnn.readNet("OpenCV/yolov3.weights", "OpenCV/yolov3.cfg")
classes = []
with open("OpenCV/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [48]:
def detect_objects(image, image_out, net, output_layers, colors):

    # Loading image
    img = cv2.imread(image)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)     
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label + ':' + str(round(confidences[i],2)), (x, y + 30), font, 0.1, color, 3)
    cv2.imwrite(image_out, img)



In [49]:
for file in os.listdir(path_input):    
    if os.path.splitext(file)[-1].lower() in [".jpg", ".jpeg", ".png" ] :    
        print(file)
        detect_objects(path_input + file, path_output + 'YOLO_' + file, net, output_layers, colors)



images.jpg
Lenna.jpg
test.jpg
test2.jpg
test3.jpg


voir soft_NMS

# Pour fermer en utilisant la croix
# A TESTER
img = cv2.imread('arvid.jpg', 0)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image', img)

while True:
    k = cv2.waitKey(30) & 0xFF
    if k == 27:         # wait for ESC key to exit
        cv2.destroyAllWindows()
    elif k == ord('s'):  # wait for 's' key to save and exit
        cv2.imwrite('arvid2.jpg', img)
        cv2.destroyAllWindows()
    if cv2.getWindowProperty("image", 0) == -1:
        break

In [12]:
confidences

[0.9857463836669922,
 0.5469236969947815,
 0.9937001466751099,
 0.9991943836212158,
 0.9925427436828613,
 0.7163336873054504]

# Détection de visages

In [26]:
# coding:latin-1
import sys, os
import cv2

  
def detect_faces(image, image_out, path_haar, show = False):
    # on charge l'image en mémoire
    img = cv2.imread(image)
    print(type(img))
    #cv2.imshow('Image',img)
    # on charge le modèle de détection des visages
    face_model = cv2.CascadeClassifier("OpenCV/" + path_haar)
     
     
    # détection du ou des visages
    faces = face_model.detectMultiScale(img)
     
    # on place un cadre autour des visages
    print ("nombre de visages", len(faces), "dimension de l'image", img.shape, "image", image)
    for face in faces:
        cv2.rectangle(img, (face[0], face[1]), (face[0] + face[2], face[0] + face[3]), (255, 0, 0), 3)
         
    # on sauvegarde le résultat final
    cv2.imwrite(image_out, img)
     
    # pour voir l'image, presser ESC pour sortir
    if show :
        cv2.imshow("visage",img)
        if cv2.waitKey(5000) == 27: cv2.destroyWindow("visage")
                                       
path_haar_frontal = 'haarcascade_frontalface_alt2.xml'
path_haar_profile = 'haarcascade_profileface.xml'


detect_faces('Images/Input/test.jpg', 'res_visage_' + 'front_' + file, path_haar_frontal, True)


<class 'numpy.ndarray'>
nombre de visages 1 dimension de l'image (878, 1170, 3) image Images/Input/test.jpg


In [31]:

for file in os.listdir(path_input):    
    if os.path.splitext(file)[-1].lower() in [".jpg", ".jpeg", ".png" ] :    
        print(file)
        detect_faces(path_input + file, path_output + 'res_visage_' + 'front_' + file, path_haar_frontal, True)

images.jpg
<class 'numpy.ndarray'>
nombre de visages 0 dimension de l'image (147, 342, 3) image Images/Input/images.jpg
Lenna.jpg
<class 'numpy.ndarray'>
nombre de visages 1 dimension de l'image (512, 512, 3) image Images/Input/Lenna.jpg
test.jpg
<class 'numpy.ndarray'>
nombre de visages 1 dimension de l'image (878, 1170, 3) image Images/Input/test.jpg
test2.jpg
<class 'numpy.ndarray'>
nombre de visages 2 dimension de l'image (878, 1170, 3) image Images/Input/test2.jpg
test3.jpg
<class 'numpy.ndarray'>
nombre de visages 0 dimension de l'image (878, 1170, 3) image Images/Input/test3.jpg


In [32]:
for file in os.listdir(path_input):    
    if os.path.splitext(file)[-1].lower() in [".jpg", ".jpeg", ".png" ] :    
        print(file)
        detect_faces(path_input + file, path_output + 'res_visage_' + 'profile_' + file, path_haar_profile, True)

images.jpg
<class 'numpy.ndarray'>
nombre de visages 1 dimension de l'image (147, 342, 3) image Images/Input/images.jpg
Lenna.jpg
<class 'numpy.ndarray'>
nombre de visages 0 dimension de l'image (512, 512, 3) image Images/Input/Lenna.jpg
test.jpg
<class 'numpy.ndarray'>
nombre de visages 1 dimension de l'image (878, 1170, 3) image Images/Input/test.jpg
test2.jpg
<class 'numpy.ndarray'>
nombre de visages 1 dimension de l'image (878, 1170, 3) image Images/Input/test2.jpg
test3.jpg
<class 'numpy.ndarray'>
nombre de visages 0 dimension de l'image (878, 1170, 3) image Images/Input/test3.jpg


# Création de features